In [1]:
import json
import openai

Creating Flask App

In [2]:
from flask import Flask, redirect, url_for, request, Response
from flask_mail import Mail, Message 

app = Flask(__name__)

@app.route("/")
def home():
    return 'Kathan Here!'

Reading Email Credentials

In [3]:
f = open("email_cred.json", "r")

cred = json.load(f)

email_address = cred.get("email_address")
email_password = cred.get("email_password")

print(cred);

{'email_address': 'kathan.naik@aethereus.com', 'email_password': 'Bits@20074985'}


Setting Email Config

In [4]:
app.config['MAIL_SERVER']='smtp.office365.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USERNAME'] = email_address
app.config['MAIL_PASSWORD'] = email_password
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USE_SSL'] = False
mail = Mail(app)

Reading Pulze Keys

In [5]:
f = open("key.txt", "r")

openai.api_key = f.readline()
openai.api_base = "https://api.pulze.ai/v1"

f.close()

Defining System Instruction

In [6]:
instructiones = '''User will give an email text as message content. Your task is to retrive email signature from that email. You should follow below steps 
1) figure out the name of person sending the mail
2) find the name of the organization or company the person belongs to if mentioned any
3) check if there is an mobile number the person writing email might have given
4) return the response message content in JSON like format as below
    {
        "name": "name that you have figured out"
        "org": "organization/company"
        "number": mobile number found
    }
5) you can keep fields empty by putting 'NULL' if you do not find any of them'''

Function to Call Pulze API

1) Signature Parsing Call

In [7]:
def pulze_llm(mail):
    chat_response = openai.ChatCompletion.create(
        model="pulze-v0",
        max_tokens=100,
        messages=[
            {"role": "system", "content": instructiones},
            {"role": "user", "content": mail}
        ],
    )

    print(chat_response)
    return chat_response.choices[0].message.content

2) Product Parsing Call

In [8]:
def pulze_llm_p(mail, list):
    
    listMsg = (
        '''Given is the list of product name and their description separated by an equal to/= sign'''
            
        + list + 
            
        '''
        1) Your task is to figure out if the user has mentioned any of the products in their message.
        2) Return the name of the product in JSON format as below:
        {
            "product": "the product" 
        }
        4) Strictly include only one product name given in the list as the product.
        3) You can keep fields empty by putting 'NULL' if you do not find any of them.
        '''
    )
    
    chat_response = openai.ChatCompletion.create(
        
        model="pulze-v0",
        max_tokens=100,
        messages=[
            {"role": "system", "content": listMsg},
            {"role": "user", "content": mail}
        ],
    )

    print(chat_response)
    return chat_response.choices[0].message.content

POST Request Handlers

1) Signature Parsing

In [9]:
@app.route("/mailpost", methods=["POST"])
def signatureParsing():
    if request.method == "POST":
        user_address = request.form["address"]
        user_email = request.form["email"]
        access_password = request.form["password"]
        
        if(access_password == "Aethereus@123"):
            msg = Message( 
                'Email-Conformation', 
                sender = email_address, 
                recipients = [user_address] 
               ) 
            msg.body = 'A case has been raised based on your email and our team will be contacting you soon. Thank you for reaching out!'
            mail.send(msg) 
            
            r = Response(response=pulze_llm(user_email), status=200, mimetype="application/json")
            r.headers["Content-Type"] = "application/json; charset=utf-8"
            return r
        else:
            return "invalid request"

2) Product Parsing

In [10]:
@app.route("/product", methods=["POST"])
def productParsing():
    if request.method == "POST":
        user_email = request.form["email"]
        product_list = request.form["products"]
        access_password = request.form["password"]
        
        if(access_password == "Aethereus@123"):
            
            r = Response(response=pulze_llm_p(user_email, product_list), status=200, mimetype="application/json")
            r.headers["Content-Type"] = "application/json; charset=utf-8"
            return r
        else:
            return "invalid request"

Running the App

In [ ]:
if __name__ == "__main__":
    app.run(host="localhost", port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8080
Press CTRL+C to quit
127.0.0.1 - - [13/Nov/2023 15:23:48] "POST /product HTTP/1.1" 200 -


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "{\n    \"product\": \"rose\"\n}",
        "role": "assistant"
      }
    }
  ],
  "created": 1699869080513,
  "id": "2cb1b12a-9c4e-4242-b0aa-5bd56a6d6352",
  "metadata": {
    "app_id": "f05910bc-7f61-4623-b323-4f7981526ecc",
    "category": "Education & Learning",
    "cost_savings": {
      "completion_tokens": 0.000522,
      "prompt_tokens": 0.024824,
      "total_tokens": 0.025346
    },
    "costs": {
      "completion_tokens": 1.8e-05,
      "prompt_tokens": 0.000856,
      "total_tokens": 0.000874
    },
    "latency": 0.823,
    "max_tokens": 100,
    "model": {
      "model": "gpt-3.5-turbo",
      "namespace": "openai/gpt-3.5-turbo",
      "provider": "openai"
    },
    "retries": 0,
    "score": 0.333,
    "scores": {
      "best_models": [
        {
          "openai/gpt-3.5-turbo": 0.333
        },
        {
          "mosaicml/llama2-70b-chat": 0.316
        }

127.0.0.1 - - [13/Nov/2023 15:24:06] "POST /product HTTP/1.1" 200 -


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "{\n   \"product\": \"NULL\"\n}",
        "role": "assistant"
      }
    }
  ],
  "created": 1699869097970,
  "id": "335eba7b-fa4d-426d-bbcc-dab1508f8408",
  "metadata": {
    "app_id": "f05910bc-7f61-4623-b323-4f7981526ecc",
    "category": "Education & Learning",
    "cost_savings": {
      "completion_tokens": 0.000522,
      "prompt_tokens": 0.024766,
      "total_tokens": 0.025288
    },
    "costs": {
      "completion_tokens": 1.8e-05,
      "prompt_tokens": 0.000854,
      "total_tokens": 0.000872
    },
    "latency": 0.701,
    "max_tokens": 100,
    "model": {
      "model": "gpt-3.5-turbo",
      "namespace": "openai/gpt-3.5-turbo",
      "provider": "openai"
    },
    "retries": 0,
    "score": 0.333,
    "scores": {
      "best_models": [
        {
          "openai/gpt-3.5-turbo": 0.333
        },
        {
          "mosaicml/llama2-70b-chat": 0.316
        }


127.0.0.1 - - [13/Nov/2023 15:24:10] "POST /product HTTP/1.1" 200 -


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "{\n    \"product\": \"rose\"\n}",
        "role": "assistant"
      }
    }
  ],
  "created": 1699869102715,
  "id": "f257a540-3935-4dbc-a941-06f3099e49de",
  "metadata": {
    "app_id": "f05910bc-7f61-4623-b323-4f7981526ecc",
    "category": "Education & Learning",
    "cost_savings": {
      "completion_tokens": 0.000522,
      "prompt_tokens": 0.024766,
      "total_tokens": 0.025288
    },
    "costs": {
      "completion_tokens": 1.8e-05,
      "prompt_tokens": 0.000854,
      "total_tokens": 0.000872
    },
    "latency": 0.507,
    "max_tokens": 100,
    "model": {
      "model": "gpt-3.5-turbo",
      "namespace": "openai/gpt-3.5-turbo",
      "provider": "openai"
    },
    "retries": 0,
    "score": 0.333,
    "scores": {
      "best_models": [
        {
          "openai/gpt-3.5-turbo": 0.333
        },
        {
          "mosaicml/llama2-70b-chat": 0.316
        }

127.0.0.1 - - [13/Nov/2023 16:18:19] "POST /product HTTP/1.1" 200 -


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "{\n    \"product\": \"rose\"\n}",
        "role": "assistant"
      }
    }
  ],
  "created": 1699872351281,
  "id": "18ee6bfc-b3c6-4d59-b9ae-3cbe6e0a10de",
  "metadata": {
    "app_id": "f05910bc-7f61-4623-b323-4f7981526ecc",
    "category": "Education & Learning",
    "cost_savings": {
      "completion_tokens": 0.000522,
      "prompt_tokens": 0.024766,
      "total_tokens": 0.025288
    },
    "costs": {
      "completion_tokens": 1.8e-05,
      "prompt_tokens": 0.000854,
      "total_tokens": 0.000872
    },
    "latency": 0.8,
    "max_tokens": 100,
    "model": {
      "model": "gpt-3.5-turbo",
      "namespace": "openai/gpt-3.5-turbo",
      "provider": "openai"
    },
    "retries": 0,
    "score": 0.333,
    "scores": {
      "best_models": [
        {
          "openai/gpt-3.5-turbo": 0.333
        },
        {
          "mosaicml/llama2-70b-chat": 0.316
        }
 